In [89]:
import pandas as pd

df = pd.read_csv('diabetes.csv')

In [90]:
df['DiabetesPedigreeFunction'].mean()

np.float64(0.47187630208333325)

In [91]:
import numpy as np
df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']] = df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']].replace(0, np.nan)

In [92]:
def median_target(var):   
    median_by_outcome = df[df[var].notnull()]
    median_by_outcome = median_by_outcome[[var, 'Outcome']].groupby(['Outcome'])[[var]].median().reset_index()
    return median_by_outcome

In [93]:
columns = df.columns
columns = columns.drop("Outcome")
for i in columns:
    median_target(i)
    df.loc[(df['Outcome'] == 0 ) & (df[i].isnull()), i] = median_target(i)[i][0]
    df.loc[(df['Outcome'] == 1 ) & (df[i].isnull()), i] = median_target(i)[i][1]

In [94]:
import pandas as pd
from sklearn.ensemble import IsolationForest

iso_forest = IsolationForest(contamination='auto', random_state=42)

df['Outlier'] = iso_forest.fit_predict(df)

In [95]:
df['IF_Score'] = iso_forest.decision_function(df.drop(columns=['Outlier']))
df = df.sort_values(by='IF_Score')

In [96]:
thres = np.percentile(df['IF_Score'], 13) 

In [97]:
thres

np.float64(-0.0009989871470563933)

In [98]:
df['Outlier'] = df['IF_Score'] < thres

In [99]:
df_clean = df[~(df['Outlier'] == -1)]

In [100]:
def klasifikasi_bmi(bmi):
    if bmi < 18.5:
        return "Underweight (bmi < 18.5)"
    elif 18.5 <= bmi <= 22.9:
        return "Normal (18.5 <= bmi <= 22.9)"
    elif 23 <= bmi <= 24.9:
        return "Overweight (23 <= bmi <= 24.9)"
    elif 25 <= bmi <= 29.9:
        return "Obesity 1 (25 <= bmi <= 29.9)"
    else:
        return "Obesity 2 (bmi > 29.9)"

df["Kategori_BMI"] = df["BMI"].apply(klasifikasi_bmi)
print(df)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
445          5.0    180.0           78.0           63.0     14.0  59.4   
228          4.0    197.0           70.0           39.0    744.0  36.7   
370          3.0    173.0           82.0           48.0    465.0  38.4   
13           1.0    189.0           60.0           23.0    846.0  30.1   
254         12.0     92.0           62.0            7.0    258.0  27.6   
..           ...      ...            ...            ...      ...   ...   
615          3.0    106.0           72.0           27.0    102.5  25.8   
233          4.0    122.0           68.0           27.0    102.5  35.0   
168          4.0    110.0           66.0           27.0    102.5  31.9   
122          2.0    107.0           74.0           30.0    100.0  33.6   
426          3.0     94.0           70.0           27.0    102.5  30.1   

     DiabetesPedigreeFunction  Age  Outcome  Outlier  IF_Score  \
445                     2.420   25        1  

In [101]:
def klasifikasi_insulin(insulin):
    if insulin <= 16 :
        return "Low (insulin <= 16)"
    elif 16 < insulin <= 166 :
        return "Normal (16 < insulin <= 166)"
    else :
        return "High (insulin > 166)"
    
df["Kategori_Insulin"] = df["Insulin"].apply(klasifikasi_insulin)
print(df)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
445          5.0    180.0           78.0           63.0     14.0  59.4   
228          4.0    197.0           70.0           39.0    744.0  36.7   
370          3.0    173.0           82.0           48.0    465.0  38.4   
13           1.0    189.0           60.0           23.0    846.0  30.1   
254         12.0     92.0           62.0            7.0    258.0  27.6   
..           ...      ...            ...            ...      ...   ...   
615          3.0    106.0           72.0           27.0    102.5  25.8   
233          4.0    122.0           68.0           27.0    102.5  35.0   
168          4.0    110.0           66.0           27.0    102.5  31.9   
122          2.0    107.0           74.0           30.0    100.0  33.6   
426          3.0     94.0           70.0           27.0    102.5  30.1   

     DiabetesPedigreeFunction  Age  Outcome  Outlier  IF_Score  \
445                     2.420   25        1  

In [102]:
def klasifikasi_BloodPressure (DPB):
    if DPB < 80 :
        return 'normal (tekanan darah diastole < 80)'
    elif 80 <= DPB <=89 :
        return 'prehipertensi (80 <= tekanan darah diastole <=89)'
    elif 90 <= DPB <= 99 :
        return 'hipertensi tingkat 1 (90 <= tekanan darah diastole <= 99)'
    else:
        return "hipertensi tingkat 2 (tekanan darah diastole > 99)"
    
df["Kategori_BloodPressure"] = df["BloodPressure"].apply(klasifikasi_BloodPressure)
print(df)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
445          5.0    180.0           78.0           63.0     14.0  59.4   
228          4.0    197.0           70.0           39.0    744.0  36.7   
370          3.0    173.0           82.0           48.0    465.0  38.4   
13           1.0    189.0           60.0           23.0    846.0  30.1   
254         12.0     92.0           62.0            7.0    258.0  27.6   
..           ...      ...            ...            ...      ...   ...   
615          3.0    106.0           72.0           27.0    102.5  25.8   
233          4.0    122.0           68.0           27.0    102.5  35.0   
168          4.0    110.0           66.0           27.0    102.5  31.9   
122          2.0    107.0           74.0           30.0    100.0  33.6   
426          3.0     94.0           70.0           27.0    102.5  30.1   

     DiabetesPedigreeFunction  Age  Outcome  Outlier  IF_Score  \
445                     2.420   25        1  

In [103]:
from sklearn.preprocessing import LabelEncoder

encoder_bmi = LabelEncoder()
df['Kategori_BMI_encoded'] = encoder_bmi.fit_transform(df['Kategori_BMI'])

In [104]:
encoder_insulin=LabelEncoder()
df['Kategori_Insulin_encoded'] = encoder_insulin.fit_transform(df['Kategori_Insulin'])

In [105]:
encoder_BP=LabelEncoder()
df['Kategori_BP_encoded'] = encoder_BP.fit_transform(df['Kategori_BloodPressure'])

In [106]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Outlier,IF_Score,Kategori_BMI,Kategori_Insulin,Kategori_BloodPressure,Kategori_BMI_encoded,Kategori_Insulin_encoded,Kategori_BP_encoded
445,5.0,180.0,78.0,63.0,14.0,59.4,2.420,25,1,True,-0.173641,Obesity 2 (bmi > 29.9),Low (insulin <= 16),normal (tekanan darah diastole < 80),2,1,2
228,4.0,197.0,70.0,39.0,744.0,36.7,2.329,31,0,True,-0.161247,Obesity 2 (bmi > 29.9),High (insulin > 166),normal (tekanan darah diastole < 80),2,0,2
370,3.0,173.0,82.0,48.0,465.0,38.4,2.137,25,1,True,-0.114415,Obesity 2 (bmi > 29.9),High (insulin > 166),prehipertensi (80 <= tekanan darah diastole <=89),2,0,3
13,1.0,189.0,60.0,23.0,846.0,30.1,0.398,59,1,True,-0.107939,Obesity 2 (bmi > 29.9),High (insulin > 166),normal (tekanan darah diastole < 80),2,0,2
254,12.0,92.0,62.0,7.0,258.0,27.6,0.926,44,1,True,-0.099899,Obesity 1 (25 <= bmi <= 29.9),High (insulin > 166),normal (tekanan darah diastole < 80),1,0,2


In [38]:
kategori = df[['Kategori_BMI', 'Kategori_Insulin', 'Kategori_BloodPressure']]
kategori.head()

,Kategori_BMI,Kategori_Insulin,Kategori_BloodPressure
445,Obesity 2 (bmi > 29.9),Low (insulin <= 16),normal (tekanan darah diastole < 80)
228,Obesity 2 (bmi > 29.9),High (insulin > 166),normal (tekanan darah diastole < 80)
370,Obesity 2 (bmi > 29.9),High (insulin > 166),prehipertensi (80 <= tekanan darah diastole <=89)
13,Obesity 2 (bmi > 29.9),High (insulin > 166),normal (tekanan darah diastole < 80)
254,Obesity 1 (25 <= bmi <= 29.9),High (insulin > 166),normal (tekanan darah diastole < 80)


In [107]:
y=df['Outcome']
X=df.drop(['Outcome','Outlier',
       'IF_Score', 'Kategori_Insulin',
       'Kategori_BloodPressure', 'Kategori_BMI'], axis=1)

In [108]:
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Kategori_BMI_encoded,Kategori_Insulin_encoded,Kategori_BP_encoded
445,5.0,180.0,78.0,63.0,14.0,59.4,2.420,25,2,1,2
228,4.0,197.0,70.0,39.0,744.0,36.7,2.329,31,2,0,2
370,3.0,173.0,82.0,48.0,465.0,38.4,2.137,25,2,0,3
13,1.0,189.0,60.0,23.0,846.0,30.1,0.398,59,2,0,2
254,12.0,92.0,62.0,7.0,258.0,27.6,0.926,44,1,0,2


In [109]:
kategori1 = df[['Kategori_BMI_encoded',	'Kategori_Insulin_encoded',	'Kategori_BP_encoded']]
kategori1.head()

,Kategori_BMI_encoded,Kategori_Insulin_encoded,Kategori_BP_encoded
445,2,1,2
228,2,0,2
370,2,0,3
13,2,0,2
254,1,0,2


In [110]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train , y_test = train_test_split(X,y, test_size=0.25, random_state=42)

In [111]:
from sklearn.preprocessing import StandardScaler
scaler =StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [112]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(class_weight='balanced', C=0.1)
model.fit(X_train, y_train)

LogisticRegression(C=0.1, class_weight='balanced')

In [113]:
y_pred = model.predict(X_test)

In [114]:
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(y_test, y_pred)
matrix

array([[110,  16],
       [  9,  57]])

In [115]:
tn, fp, fn, tp = matrix.ravel()

print(f'TN(True Negative): {tn}')
print(f'FP(False Positive): {fp}')
print(f'FN(False Negative): {fn}')
print(f'TP(True Positive): {tp}')

TN(True Negative): 110
FP(False Positive): 16
FN(False Negative): 9
TP(True Positive): 57


In [116]:
import numpy as np

In [117]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy = {accuracy*100: .2f}%')

Accuracy =  86.98%


In [118]:
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred)
print(f'Precision = {precision*100: .2f}%')

Precision =  78.08%


In [119]:
from sklearn.metrics import recall_score

recall = recall_score(y_test, y_pred)
print(f'Recall = {recall*100: .2f}%')

Recall =  86.36%


In [120]:
from sklearn.metrics import f1_score

F1 = f1_score(y_test, y_pred)
print(f'F1 = {F1*100: .2f}%')

F1 =  82.01%


In [121]:
Average_Metrics = (accuracy + precision + recall + F1)/4
print(f'Average Metrics = {Average_Metrics*100:.2f}%')

Average Metrics = 83.36%


In [54]:
import joblib

In [55]:
joblib.dump(encoder_bmi, 'encoder_bmi.pkl')
joblib.dump(encoder_insulin, 'encoder_insulin.pkl')
joblib.dump(encoder_BP, 'encoder_BP.pkl')
joblib.dump(model, 'Diabetes_Prediction.pkl')
joblib.dump(scaler, 'Scaler_Data.pkl')

['Scaler_Data.pkl']